In [42]:
import requests
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
from lxml import etree

In [43]:
jugador = "Lamine Yamal"
fecha_inicio = "2022-01-01"
fecha_fin = "2023-01-31"

url = f"https://news.google.com/search?q={jugador.replace(' ', '%20')}+after:{fecha_inicio}+before:{fecha_fin}"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

elements = soup.find_all('c-wiz', {'jsrenderer': 'ARwRbe'})

In [44]:
l_href = []

for element in elements:
    links = element.find_all('a')

    for link in links:
        href = link.get('href')

        if href:
            l_href.append(href)

print(l_href)


['https://support.google.com/googlenews?p=consumer_info&hl=en', './read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmpMaDRGU25MQkVzbDA3eXhfQ1AydGR1cFk4RVlxazlzR1JvaXBoVmpUT0tNZXNnX2ZPUEJJS3BmdmxJTGhfdS1KS0UyNzFVRnZhY250eFo1dkROZ0NWb0RzN25OY1JyM0k0OGRwVkJuS1VrRkhSc1FnMGVFQzFwNGtLMA?hl=en-US&gl=US&ceid=US%3Aen', './read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmp

In [45]:
def get_google_params(url):
    response = requests.get(url, headers={})
    print(response.status_code)
    tree = etree.HTML(response.text)

    sign = tree.xpath('//c-wiz/div/@data-n-a-sg')[0]
    ts = tree.xpath('//c-wiz/div/@data-n-a-ts')[0]
    source = tree.xpath('//c-wiz/div/@data-n-a-id')[0]
    return source, sign, ts




def get_origin_url(source, sign, ts):
    url = f"https://news.google.com/_/DotsSplashUi/data/batchexecute"
    req_data = [[[
        "Fbv4je",
        f"[\"garturlreq\",[[\"zh-HK\",\"HK\",[\"FINANCE_TOP_INDICES\",\"WEB_TEST_1_0_0\"],null,null,1,1,\"HK:zh-Hant\",null,480,null,null,null,null,null,0,5],\"zh-HK\",\"HK\",1,[2,4,8],1,1,null,0,0,null,0],\"{source}\",{ts},\"{sign}\"]",
        None,
        "generic"
    ]]]
    payload = f"f.req={quote(json.dumps(req_data))}"
    headers = {
      'Host': 'news.google.com',
      # 'Content-Length': '712',
      'X-Same-Domain': '1',
      'Accept-Language': 'zh-CN',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:109.0) Gecko/20100101 Firefox/115.0',
      'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8',
      'Accept': '*/*',
      'Origin': 'https://news.google.com',
      # 'X-Client-Data': 'CPz5ygE=',
      'Sec-Fetch-Site': 'same-origin',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Dest': 'empty',
      'Referer': 'https://news.google.com/',
      'Accept-Encoding': 'gzip, deflate, br',
    }
    response = requests.post(url, headers=headers, data=payload)
    return response.text

In [46]:
# request to each link and extract the <p> tags
final_links = []
for link in l_href[1:]:
    absolute_link = urljoin(url, link)
    
    print(absolute_link)

    
    source, sign, ts = get_google_params(absolute_link)
    final_links.append(get_origin_url(source, sign, ts))



https://news.google.com/read/CBMivgFBVV95cUxOTE5rXzJNTl9PcEFXME41UzJwek1TMTFoeDllSlZsb2JuWjFvM3c1cUFCVDNlUF93TUt2cWNjNGlKWUloVm9acmdNbnAwWWxXZXkweFYzSkJoalNJdW9Ka09FUlR3TFlWVS1TM0hLRnFSRzIxOG1zcER4QV82RmxGR3djVFptMXJpZnpxcVYxQUQ4RkZEWXg0dWNGTExla2JCRURUeE92NVBybUNjZG9PZUhDd2hCMWZRMUlVbEJB0gHDAUFVX3lxTE5BM3NmRUVvR1pjTzJxeS16TkZrSjQ4cDJzcldMVUZFNTBLNnFfWnpKWENWVk9lRXAySVpaMHB5bXMzN3JxeFdVNmpMaDRGU25MQkVzbDA3eXhfQ1AydGR1cFk4RVlxazlzR1JvaXBoVmpUT0tNZXNnX2ZPUEJJS3BmdmxJTGhfdS1KS0UyNzFVRnZhY250eFo1dkROZ0NWb0RzN25OY1JyM0k0OGRwVkJuS1VrRkhSc1FnMGVFQzFwNGtLMA?hl=en-US&gl=US&ceid=US%3Aen
429


IndexError: list index out of range

In [31]:
from googlenewsdecoder import new_decoderv1

def main():

    interval_time = 5 # default interval is 1 sec, if not specified

    source_url = "https://news.google.com/read/CBMimAFBVV95cUxPX2lma2ZTaDNpR3VQN085R3VoZk1kWGxYTkdTMmZjYWY1d25PUGZsMzFFZ2dIRGlFYUJFeGxSaDdnTUtIdGRzNi1fT0tBQW9ETjVla1NjQUUxTHNtRWlOWEQ2emJQMWRENFZGS3E2U01IVWV0eDBpYUZLb0FoM1BLbWdBbW1RZ0hsTExuazB5TzRnS1RvWnJxbtIBjwFBVV95cUxQeFJrOEN3WUpwZ0hyTHhtRDk3cUNvV05KYlF6ZlhuUk0yVllsVlhlT1dYSExCcUlxdEdxX3hOdlItcGVYY18xM2otS1FVejJNU1pUbm9odVRvX3V1VkIzQlFVTm91cGRsYlJmUDJJeUlId3libDYwcHI0eWtFZGdMS0NJY2hZVEtWNW5uXzBzQQ?hl=en-US&gl=US&ceid=US%3Aen"

    try:
        decoded_url = new_decoderv1(source_url, interval=interval_time)
        if decoded_url.get("status"):
            print("Decoded URL:", decoded_url["decoded_url"])
        else:
            print("Error:", decoded_url["message"])
    except Exception as e:
        print(f"Error occurred: {e}")

    # Output: decoded_urls - [{'status': True, 'decoded_url': 'https://healthdatamanagement.com/articles/empowering-the-quintuple-aim-embracing-an-essential-architecture/'}]

if __name__ == "__main__":
    main()

2024-09-30 15:07:30,260 - root - ERROR - get_decoding_params Request error: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://news.google.com/articles/CBMimAFBVV95cUxPX2lma2ZTaDNpR3VQN085R3VoZk1kWGxYTkdTMmZjYWY1d25PUGZsMzFFZ2dIRGlFYUJFeGxSaDdnTUtIdGRzNi1fT0tBQW9ETjVla1NjQUUxTHNtRWlOWEQ2emJQMWRENFZGS3E2U01IVWV0eDBpYUZLb0FoM1BLbWdBbW1RZ0hsTExuazB5TzRnS1RvWnJxbtIBjwFBVV95cUxQeFJrOEN3WUpwZ0hyTHhtRDk3cUNvV05KYlF6ZlhuUk0yVllsVlhlT1dYSExCcUlxdEdxX3hOdlItcGVYY18xM2otS1FVejJNU1pUbm9odVRvX3V1VkIzQlFVTm91cGRsYlJmUDJJeUlId3libDYwcHI0eWtFZGdMS0NJY2hZVEtWNW5uXzBzQQ&q=EgSKBOPMGJHF6rcGIjBn6XZKyIFPcDWeo1RS5C0-qJazzwtuN-K7hZSAKTVB3xvuI4HmaGe0V6E8oOJRAw4yAXJaAUM


Error: Request error in get_decoding_params: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://news.google.com/articles/CBMimAFBVV95cUxPX2lma2ZTaDNpR3VQN085R3VoZk1kWGxYTkdTMmZjYWY1d25PUGZsMzFFZ2dIRGlFYUJFeGxSaDdnTUtIdGRzNi1fT0tBQW9ETjVla1NjQUUxTHNtRWlOWEQ2emJQMWRENFZGS3E2U01IVWV0eDBpYUZLb0FoM1BLbWdBbW1RZ0hsTExuazB5TzRnS1RvWnJxbtIBjwFBVV95cUxQeFJrOEN3WUpwZ0hyTHhtRDk3cUNvV05KYlF6ZlhuUk0yVllsVlhlT1dYSExCcUlxdEdxX3hOdlItcGVYY18xM2otS1FVejJNU1pUbm9odVRvX3V1VkIzQlFVTm91cGRsYlJmUDJJeUlId3libDYwcHI0eWtFZGdMS0NJY2hZVEtWNW5uXzBzQQ&q=EgSKBOPMGJHF6rcGIjBn6XZKyIFPcDWeo1RS5C0-qJazzwtuN-K7hZSAKTVB3xvuI4HmaGe0V6E8oOJRAw4yAXJaAUM
